In [1]:
from env import conn
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline
import datetime as dt
import plotly.express as px


In [2]:
def return_df(query):
    cs = conn.cursor()
    query = query
    result = cs.execute(query)
    df = result.fetch_pandas_all()
    cs.close()
    return df


# Investigating launch for Stonegate
We will be using K-means to categorise users via clustering. Following advice from this page into reatiler customer data and using the RMF process and beyond.
https://medium.com/analytics-vidhya/customer-segmentation-kmeans-clustering-7670e317b54c

In [3]:
tables = return_df(
    """
    SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE'
order by table_schema, table_name;    
    """
        )
tables


,TABLE_SCHEMA,TABLE_NAME
0,JIRA,JIRA__SPRINT__SPRINT
1,JIRA,JIRA__TEAM__SPRINT
2,LOYALTY_CARDS,LC201__LOYALTY_CARD_JOURNEY_FUNNEL__USER_LEVEL...
3,LOYALTY_CARDS,LC__ERRORS__DAILY_STATUS_ROLLUP_USER_LEVEL
4,LOYALTY_CARDS,LC__LINKS_JOINS__DAILY_RETAILER
5,LOYALTY_CARDS,LC__LINKS_JOINS__DAILY_RETAILER_CHANNEL
6,LOYALTY_CARDS,LC__LINKS_JOINS__DAILY_RETAILER_CHANNEL__FORECAST
7,LOYALTY_CARDS,LC__LINKS_JOINS__MONTHLY_RETAILER
8,LOYALTY_CARDS,LC__LINKS_JOINS__MONTHLY_RETAILER_CHANNEL
9,LOYALTY_CARDS,LC__LINKS_JOINS__MONTHLY_RETAILER_CHANNEL__GROWTH


### Generating dataframnes for analysis

In [4]:
lc_daily_metrics = return_df(
    """select * from METRICS.LOYALTY_CARDS.LC__LINKS_JOINS__DAILY_RETAILER
        where loyalty_plan_company = 'Stonegate Group'
        order by date;    
    """
        )

txn_daily_metrics = return_df(
    """select * from METRICS.TRANSACTIONS.TRANS__TRANS__DAILY_RETAILER
        where loyalty_plan_company = 'Stonegate Group'
         order by date;    
    """
        )

user_level_txns = return_df(
    """select * from PROD.BINK_SECURE.FACT_TRANSACTION
        where loyalty_plan_company = 'Stonegate Group'
         order by event_date_time;    
    """
        )


## Categorising users based on txns


### Inspecting and Cleaning data

In [5]:
user_level_txns.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163613 entries, 0 to 163612
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   EVENT_ID               163613 non-null  int32         
 1   EVENT_DATE_TIME        163613 non-null  datetime64[ns]
 2   USER_ID                163613 non-null  object        
 3   EXTERNAL_USER_REF      163613 non-null  object        
 4   CHANNEL                163613 non-null  object        
 5   BRAND                  163613 non-null  object        
 6   TRANSACTION_ID         163613 non-null  object        
 7   PROVIDER_SLUG          163613 non-null  object        
 8   FEED_TYPE              0 non-null       object        
 9   DUPLICATE_TRANSACTION  163613 non-null  bool          
 10  LOYALTY_PLAN_NAME      163613 non-null  object        
 11  LOYALTY_PLAN_COMPANY   163613 non-null  object        
 12  TRANSACTION_DATE       163613 non-null  date

### Creating features for analysis

In [6]:
#hash external_user_ref
user_level_txns["EXTERNAL_USER_REF"] = user_level_txns["EXTERNAL_USER_REF"].apply(hash)

user_level_txns_agg = user_level_txns.groupby(by="EXTERNAL_USER_REF", ).agg(
    TRANSACTION_ID_count=('TRANSACTION_ID', 'count'),
    SPEND_AMOUNT_sum = ('SPEND_AMOUNT','sum'),
    SPEND_AMOUNT_mean = ('SPEND_AMOUNT','mean'),
    SPEND_AMOUNT_min = ('SPEND_AMOUNT','min', ),
    SPEND_AMOUNT_max = ('SPEND_AMOUNT','max', ),
    SPEND_AMOUNT_range = ('SPEND_AMOUNT',lambda x: x.max() - x.min()),
    EVENT_DATE_TIME_range = ('EVENT_DATE_TIME', lambda x: (x.max() - x.min()).days + 1),
    EVENT_DATE_TIME_last = ('EVENT_DATE_TIME', lambda x: (pd.to_datetime('today').normalize() - x.max()).days + 1),
    EVENT_DATE_TIME_max = ('EVENT_DATE_TIME', 'max'),
    EVENT_DATE_TIME_min = ('EVENT_DATE_TIME', 'min')
)

user_level_txns_agg['txn_frequency'] = user_level_txns_agg['TRANSACTION_ID_count']/user_level_txns_agg['EVENT_DATE_TIME_range']
user_level_txns_agg['spend_adj'] = user_level_txns_agg['SPEND_AMOUNT_sum']/user_level_txns_agg['EVENT_DATE_TIME_range']



### Further feature selection
ATV, modal txn value, time of day, stores visited?, 

In [7]:

user_level_txns_agg['txn_frequency'] = user_level_txns_agg['TRANSACTION_ID_count']/user_level_txns_agg['EVENT_DATE_TIME_range']
user_level_txns_agg['spend_adj'] = user_level_txns_agg['SPEND_AMOUNT_sum']/user_level_txns_agg['EVENT_DATE_TIME_range']

In [8]:
user_level_txns_agg.info()


<class 'pandas.core.frame.DataFrame'>
Index: 25361 entries, -9222749140742301109 to 9223301789136707646
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   TRANSACTION_ID_count   25361 non-null  int64         
 1   SPEND_AMOUNT_sum       25361 non-null  float64       
 2   SPEND_AMOUNT_mean      25361 non-null  float64       
 3   SPEND_AMOUNT_min       25361 non-null  float64       
 4   SPEND_AMOUNT_max       25361 non-null  float64       
 5   SPEND_AMOUNT_range     25361 non-null  float64       
 6   EVENT_DATE_TIME_range  25361 non-null  int64         
 7   EVENT_DATE_TIME_last   25361 non-null  int64         
 8   EVENT_DATE_TIME_max    25361 non-null  datetime64[ns]
 9   EVENT_DATE_TIME_min    25361 non-null  datetime64[ns]
 10  txn_frequency          25361 non-null  float64       
 11  spend_adj              25361 non-null  float64       
dtypes: datetime64[ns](2), float64(7)

In [9]:
user_level_txns_agg.describe()


,TRANSACTION_ID_count,SPEND_AMOUNT_sum,SPEND_AMOUNT_mean,SPEND_AMOUNT_min,SPEND_AMOUNT_max,SPEND_AMOUNT_range,EVENT_DATE_TIME_range,EVENT_DATE_TIME_last,EVENT_DATE_TIME_max,EVENT_DATE_TIME_min,txn_frequency,spend_adj
count,25361.000000,25361.000000,25361.000000,25361.000000,25361.000000,25361.000000,25361.000000,25361.000000,25361,25361,25361.000000,25361.000000
mean,6.451362,48.947906,9.278581,6.500668,14.095311,7.594643,7.067229,8.265526,2023-11-26 04:27:50.661200128,2023-11-19 20:34:18.621010688,1.251867,11.459350
min,1.000000,0.050000,0.050000,0.010000,0.050000,0.000000,1.000000,1.000000,2023-11-06 02:49:54.341980,2023-10-28 04:03:13.085397,0.076923,0.050000
25%,1.000000,10.500000,4.721818,2.750000,6.100000,0.000000,1.000000,3.000000,2023-11-22 06:41:55.759368960,2023-11-14 00:33:24.979885056,0.631579,3.550000
50%,3.000000,23.100000,7.000000,4.300000,10.850000,4.030000,3.000000,6.000000,2023-11-28 03:47:13.557680128,2023-11-19 16:06:47.918955008,1.000000,6.857143
75%,7.000000,52.950000,10.925000,7.300000,16.850000,10.300000,12.000000,12.000000,2023-12-01 08:35:23.020625920,2023-11-25 09:55:04.455975936,1.500000,13.594444
max,186.000000,2025.500000,830.500000,830.500000,1861.620000,1859.120000,29.000000,28.000000,2023-12-03 21:32:20.729894,2023-12-03 18:54:17.028986,14.000000,830.500000
std,11.079812,80.940149,12.224467,11.479138,21.596732,19.087614,7.611641,6.149947,NaN,NaN,1.049692,17.912738


In [10]:
vars = list(user_level_txns_agg.columns)

In [11]:
#dropping data points to do RFM
user_level_txns_agg_reduced = user_level_txns_agg[['spend_adj', 'txn_frequency', 'EVENT_DATE_TIME_last']]


In [12]:
px.box(user_level_txns_agg_reduced.melt(), y='value', facet_col='variable').update_yaxes(matches=None)


### Remove outliers

In [13]:
outlier_vars = list(user_level_txns_agg_reduced.columns)
for column in outlier_vars:
    
    lower_quartile = user_level_txns_agg_reduced[column].quantile(0.25)
    print(column + "LQ" + str(lower_quartile))
    upper_quartile = user_level_txns_agg_reduced[column].quantile(0.75)
    print(column + "UQ" + str(upper_quartile))
    iqr = upper_quartile - lower_quartile
    iqr_extended = iqr * 1.5
    min_border = lower_quartile - iqr_extended
    max_border = upper_quartile + iqr_extended
    
    outliers = user_level_txns_agg_reduced[(user_level_txns_agg_reduced[column] < min_border) |     (user_level_txns_agg_reduced[column] > max_border)].index
    print(f"{len(outliers)} outliers detected in column {column}")
    
    user_level_txns_agg_reduced.drop(outliers, inplace=True)

spend_adjLQ3.55
spend_adjUQ13.594444444444443
1926 outliers detected in column spend_adj
txn_frequencyLQ0.5882352941176471
txn_frequencyUQ1.2222222222222223
1607 outliers detected in column txn_frequency
EVENT_DATE_TIME_lastLQ3.0
EVENT_DATE_TIME_lastUQ12.0
244 outliers detected in column EVENT_DATE_TIME_last


/var/folders/hd/jfhhr4n542b9500brd9s6j680000gp/T/ipykernel_4196/1469723701.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/hd/jfhhr4n542b9500brd9s6j680000gp/T/ipykernel_4196/1469723701.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/hd/jfhhr4n542b9500brd9s6j680000gp/T/ipykernel_4196/1469723701.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
px.box(user_level_txns_agg_reduced.melt(), y='value', facet_col='variable').update_yaxes(matches=None)

### Creating an algorithm
The goal is to categorise our users based on their features
For this we will explore a simple Model called K-means clustering

In [15]:
import sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score




##### Scale the data

In [16]:
#Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_level_txns_agg_reduced)
scaled_data


array([[ 0.35030666,  2.05427312,  0.89765165],
       [-0.48039574,  0.10305055,  1.76899819],
       [-0.84847119,  0.10305055, -1.19358005],
       ...,
       [-0.4601347 ,  0.10305055, -0.49650282],
       [ 0.96320294,  0.10305055,  0.72338234],
       [-0.90756587, -0.7331877 , -1.19358005]])

##### Fit model and produce outcome

In [17]:
#tune the model
kmeans = KMeans(n_clusters=10)  # Specify the number of clusters
user_level_txns_agg_reduced['cluster'] = kmeans.fit_predict(scaled_data)
user_level_txns_agg_reduced['cluster'].value_counts().sort_index()


/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/var/folders/hd/jfhhr4n542b9500brd9s6j680000gp/T/ipykernel_4196/1658574344.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



cluster
0    2703
1    5172
2    1198
3    1547
4    5008
5     862
6    1038
7    1529
8    1507
9    1020
Name: count, dtype: int64

##### Iterate through number of clusters to fine error value and plot

In [18]:
dist_df = []
for num_clusters in range (1,21):
    km = KMeans(n_clusters=num_clusters)
    km.fit(user_level_txns_agg_reduced)
    dist_df.append([num_clusters, km.inertia_])
dist_df = pd.DataFrame(dist_df, columns=["number_of_clusters", "distance"])
fig = px.line(dist_df, x="number_of_clusters", y="distance", title='Error vs Cluster')
fig.show()


/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Use

In [19]:
silhouette_avg = []
for num_clusters in range (2,11):
    km = KMeans(n_clusters=num_clusters)
    km.fit(user_level_txns_agg_reduced)
    cluster_labels = km.labels_
    silhouette_avg.append([num_clusters, silhouette_score(user_level_txns_agg_reduced, cluster_labels)])
silhouette_avg = pd.DataFrame(silhouette_avg, columns=["number_of_clusters", "silohouette_score"])


/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/Use

In [20]:
silhouette_avg


,number_of_clusters,silohouette_score
0,2,0.410071
1,3,0.432424
2,4,0.409340
3,5,0.386372
4,6,0.372856
5,7,0.372968
6,8,0.354479
7,9,0.345669
8,10,0.332582


In [21]:
fig = px.line(silhouette_avg, x="number_of_clusters", y="silohouette_score", title='silohouette_score vs Cluster')
fig.show()


#### Adjust clustering and re run

In [24]:
#tune the model
kmeans = KMeans(n_clusters=3)  # Specify the number of clusters
user_level_txns_agg_reduced['cluster'] = kmeans.fit_predict(scaled_data)
user_level_txns_agg_reduced['cluster'].value_counts().sort_index()


/Users/anandbhakta/python/snowdrops/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/var/folders/hd/jfhhr4n542b9500brd9s6j680000gp/T/ipykernel_4196/2867941401.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



cluster
0     5177
1     5446
2    10961
Name: count, dtype: int64

In [26]:
fig = px.scatter_3d(user_level_txns_agg_reduced, x='spend_adj', y='txn_frequency', z='EVENT_DATE_TIME_last',
              color='cluster')
fig.show()

In [30]:
analysis = user_level_txns_agg_reduced.groupby('cluster').agg({
    'EVENT_DATE_TIME_last':['mean','max','min'],
    'txn_frequency':['mean','max','min'],
    'spend_adj':['mean','max','min','count']})
analysis

EVENT_DATE_TIME_last         txn_frequency                      \
                        mean max min          mean       max       min   
cluster                                                                  
0                   7.960981  25   1      1.574985  2.166667  0.384615   
1                  14.950055  25   8      0.909998  2.000000  0.133333   
2                   4.268041  12   1      0.669148  1.705882  0.076923   

         spend_adj                              
              mean        max       min  count  
cluster                                         
0        15.253642  28.650000  0.300000   5177  
1         6.284542  23.000000  0.050000   5446  
2         4.491474  20.165333  0.057692  10961

In [29]:
fig = px.imshow(analysis)
fig.show()